In [1]:
%load_ext autoreload
%autoreload 2

In [17]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
from scipy.spatial.distance import cosine
import scipy
from ggplot import *


import object_class as oc
from object_vectors import *
import compare_vectors as cv

In [10]:
wordvecs = oc.wordvecs(seedlings_categories, "../../../../object_centrality/data_local/dict_glove_42b_300", 
                                             "../../../../object_centrality/data_local/glove_unit_42b_300.npy")

In [11]:
wordvecs.sort(key=lambda x: x[0])
columns = [x[0] for x in wordvecs]

pairwise_matrix = []
for vec in wordvecs:
    result = []
    for other in wordvecs:
        result.append(cosine(vec[1], other[1]))
    pairwise_matrix.append(result)
# pairwise_word = pd.DataFrame(pairwise_matrix, columns=columns, index=columns)
# pairwise_word.to_csv("pairwise_word_distances.csv")

In [13]:
image_vecs = []

for x in seedlings_categories:
    result = oc.find_central_vector(x)
    image_vecs.append((x, result[0]))
    
image_vecs.sort(key=lambda x: x[0])
pairwise_image_matrix = []
for vec in image_vecs:
    result = []
    for other in image_vecs:
        result.append(cosine(vec[1], other[1]))
    pairwise_image_matrix.append(result)
    
# columns = ['word', 'other_word', 'cosine']

# pairwise_image = pd.DataFrame(pairwise_image_matrix, columns=columns, index=columns)
# pairwise_image.to_csv("pairwise_image_distances.csv")

In [38]:
results = []
for idx1, word1 in enumerate(sorted(seedlings_categories)):
    img_vec1 = image_vecs[idx1]
    wrd_vec1 = wordvecs[idx1]
    for idx2, word2 in enumerate(sorted(seedlings_categories)):
        img_vec2 = image_vecs[idx2]
        wrd_vec2 = wordvecs[idx2]
        if word1 != word2:
            results.append([word1, word2, cosine(img_vec1[1], img_vec2[1]), cosine(wrd_vec1[1], wrd_vec2[1])])
        
pairwise = pd.DataFrame(results, columns=["X1", "word", "img_cosine", "word_cosine"])
pairwise.to_csv("pairwise_distances.csv", index=False)

In [71]:
datalong_word = pd.read_csv("data_local/datalong_word.csv").query("(X1 != word)")
datalong_image = pd.read_csv("data_local/datalong_image.csv").query("(X1 != word)")

In [104]:
def top_n_closest(df, n):
    top_n = pd.DataFrame()
    for name, group in df.groupby("X1"):
        group = group.sort_values(by="cos_dist")
        group["zscore"] = scipy.stats.zscore(group.cos_dist)
        top_n = top_n.append(group[:n])
    return top_n

In [28]:
wordvecs[0][1].shape

(300,)

In [25]:
wordvecs

[('apple',
  array([-9.62907871e-03, -9.04268294e-04, -2.13676898e-02,  2.36224841e-02,
          8.88883955e-02, -1.30598982e-01, -3.48566580e-01,  3.24352104e-02,
          3.24563529e-02, -1.61168670e-01,  1.72622975e-02,  1.79202050e-02,
          1.45672400e-01,  6.08968587e-03,  1.72784654e-02,  8.05644360e-02,
         -4.20899089e-02, -1.19153382e-01, -5.08118464e-02, -2.66384115e-02,
         -1.99921782e-03, -9.52174900e-03,  4.11932146e-02,  4.19705155e-02,
          1.10171515e-02, -2.79492516e-02,  3.81760436e-02,  2.64692708e-02,
         -5.50789172e-03,  3.44275879e-02,  4.28858650e-02,  2.20678824e-02,
          3.17325304e-02, -1.53793639e-02,  4.11024259e-02,  6.10249579e-02,
         -2.12259101e-02,  4.66952610e-02,  2.67826785e-02,  4.42265535e-02,
          8.31873601e-03, -1.04635889e-01,  1.23557258e-02, -7.94177618e-02,
         -9.09640997e-02, -3.63304204e-03,  1.24083335e-02,  1.59415079e-02,
          6.44910979e-02, -8.08641632e-03, -2.81955005e-02,  1.23

In [3]:
df = pd.read_csv("pairwise_distances.csv").drop(columns=["top7img"])

In [4]:
animate = ["puppy", "duck", "elephant", "pig", "monkey", "giraffe",
             "bear", "baby", "fish", "frog", "cow", "dog", "cat"]

In [5]:
df["animate"] = "mixed"

In [6]:
df.loc[df["word"].isin(animate) & df["X1"].isin(animate), "animate"] = "animate"

In [7]:
df.loc[~df["word"].isin(animate) & ~df["X1"].isin(animate), "animate"] = "inanimate"

In [8]:
# df.to_csv("pairwise_distance.csv", index=False)

In [9]:
df2 = pd.DataFrame()

for idx, group in df.groupby("word"):
    group["zscore_img"] = scipy.stats.zscore(group.cos_img)
    group["zscore_word"] = scipy.stats.zscore(group.cos_word)
    df2 = df2.append(group)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [10]:
overlaps = df2.query("(zscore_word <= -1) & (zscore_img <= -1)")
overlaps.to_csv("overlapping_pairwise.csv", index=False)

In [107]:
words = {}

for idx, item in df2.iterrows():
    avg_zscore = round(float((item.zscore_img+item.zscore_word))/2, 2)
    if item.word not in words:
        if item.zscore_img <= -1:
            words[item.word] = [[(item.X1, avg_zscore)],[]]
        if item.zscore_word <= -1:
            words[item.word] = [[],[(item.X1, avg_zscore)]]
    else:
        if item.zscore_img <= -1:
            words[item.word][0].append((item.X1, avg_zscore))
        if item.zscore_word <= -1:
            words[item.word][1].append((item.X1, avg_zscore))

In [110]:
overlap_ratios = {}
for key, values in words.items():
    intersect = set(values[0]) & set(values[1])
    n = len(intersect)
    overlap_ratios[key] = [round(float(n)/len(set(values[0]+values[1])), 3), 
                           [list(x) + [True] for x in intersect] +
                           [list(x) + [False] for x in values[0]+values[1] if x not in intersect]]
    

In [111]:
overlap_ratios_list = [(x, y[0], sorted(list(y[1]), key=lambda x: x[1])) for x, y in overlap_ratios.items()]

In [112]:
overlap_table = pd.DataFrame(overlap_ratios_list, columns = ["word", "overlap_ratio", "neighbors"])

In [113]:
overlap_ratios_list

[('apple',
  0.143,
  [['egg', -1.18, True],
   ['milk', -0.66, False],
   ['car', -0.6, False],
   ['puppy', -0.31, False],
   ['book', -0.24, False],
   ['bear', 1.04, False]]),
 ('fish',
  0.2,
  [['duck', -1.22, True],
   ['pig', -1.16, True],
   ['cow', -1.15, False],
   ['frog', -1.09, False],
   ['egg', -0.91, False],
   ['water', -0.72, False],
   ['monkey', -0.45, False],
   ['giraffe', 0.27, False]]),
 ('pig',
  0.111,
  [['cow', -1.7, False],
   ['monkey', -1.28, True],
   ['duck', -1.09, False],
   ['puppy', -1.06, False],
   ['fish', -0.98, False],
   ['frog', -0.8, False],
   ['dog', -0.59, False],
   ['train', -0.08, False]]),
 ('duck',
  0.125,
  [['cow', -1.41, False],
   ['pig', -1.41, False],
   ['fish', -1.31, True],
   ['egg', -1.24, False],
   ['frog', -0.95, False],
   ['elephant', -0.76, False],
   ['giraffe', -0.2, False]]),
 ('chair',
  0.0,
  [['baby', -1.58, False],
   ['ball', -0.96, False],
   ['dog', -0.85, False],
   ['frog', -0.79, False],
   ['cat', -0

In [102]:
words

{'apple': [[('egg', -1.18), ('puppy', -0.31)],
  [('bear', 1.04),
   ('book', -0.24),
   ('car', -0.6),
   ('egg', -1.18),
   ('milk', -0.66)]],
 'baby': [[('cow', -0.79),
   ('elephant', -0.96),
   ('frog', -0.87),
   ('pig', -0.64),
   ('train', -0.17)],
  [('bear', -0.06), ('dog', -1.04), ('monkey', -1.03), ('puppy', -1.15)]],
 'ball': [[('cow', -0.4), ('fish', -0.71), ('frog', -0.71), ('spoon', -0.43)],
  [('dog', -0.97), ('monkey', -0.91)]],
 'bear': [[('cow', -1.19),
   ('elephant', -1.4),
   ('monkey', -1.57),
   ('puppy', -1.47)],
  [('baby', -0.94),
   ('cat', -0.19),
   ('dog', -0.48),
   ('duck', -0.49),
   ('elephant', -1.4),
   ('frog', -1.08),
   ('monkey', -1.57)]],
 'block': [[('cow', -0.45),
   ('duck', -0.57),
   ('egg', -1.19),
   ('fish', -1.0),
   ('puppy', 0.07),
   ('train', -0.89)],
  [('ball', -0.59), ('truck', -0.43), ('water', -1.04)]],
 'book': [[('fish', -0.65),
   ('monkey', -0.57),
   ('pig', -0.55),
   ('puppy', -0.17)],
  [('baby', -1.25),
   ('car', -1

In [103]:
overlap_ratios

{'apple': [0.143,
  {('egg', -1.18)},
  [('puppy', -0.31),
   ('bear', 1.04),
   ('book', -0.24),
   ('car', -0.6),
   ('milk', -0.66)]],
 'baby': [0.0,
  set(),
  [('cow', -0.79),
   ('elephant', -0.96),
   ('frog', -0.87),
   ('pig', -0.64),
   ('train', -0.17),
   ('bear', -0.06),
   ('dog', -1.04),
   ('monkey', -1.03),
   ('puppy', -1.15)]],
 'ball': [0.0,
  set(),
  [('cow', -0.4),
   ('fish', -0.71),
   ('frog', -0.71),
   ('spoon', -0.43),
   ('dog', -0.97),
   ('monkey', -0.91)]],
 'bear': [0.182,
  {('elephant', -1.4), ('monkey', -1.57)},
  [('cow', -1.19),
   ('puppy', -1.47),
   ('baby', -0.94),
   ('cat', -0.19),
   ('dog', -0.48),
   ('duck', -0.49),
   ('frog', -1.08)]],
 'block': [0.0,
  set(),
  [('cow', -0.45),
   ('duck', -0.57),
   ('egg', -1.19),
   ('fish', -1.0),
   ('puppy', 0.07),
   ('train', -0.89),
   ('ball', -0.59),
   ('truck', -0.43),
   ('water', -1.04)]],
 'book': [0.0,
  set(),
  [('fish', -0.65),
   ('monkey', -0.57),
   ('pig', -0.55),
   ('puppy', 

In [114]:
overlap_table.sort_values(by="word").to_csv("overlap_table.csv", index=False)

In [115]:
overlap_table

,word,overlap_ratio,neighbors
0,apple,0.143,"[[egg, -1.18, True], [milk, -0.66, False], [ca..."
1,fish,0.200,"[[duck, -1.22, True], [pig, -1.16, True], [cow..."
2,pig,0.111,"[[cow, -1.7, False], [monkey, -1.28, True], [d..."
3,duck,0.125,"[[cow, -1.41, False], [pig, -1.41, False], [fi..."
4,chair,0.000,"[[baby, -1.58, False], [ball, -0.96, False], [..."
5,milk,0.167,"[[bottle, -2.12, False], [water, -2.0, True], ..."
6,monkey,0.273,"[[cow, -1.49, True], [frog, -1.43, True], [pig..."
7,cup,0.125,"[[egg, -1.32, True], [spoon, -1.25, False], [m..."
8,frog,0.091,"[[monkey, -1.53, True], [cow, -1.21, False], [..."
9,book,0.000,"[[car, -1.38, False], [baby, -1.25, False], [d..."
